In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import pickle
import requests
import os
import pathlib

In [2]:
driver = webdriver.Chrome('../../chromedriver')

In [3]:
whitelist = ['minimal', 'landscape', 'abstract']
blacklist = ['phone']
WEBSITE_PAUSE_TIME = 1
IMAGE_PAUSE_TIME = 0.5
SCROLL_PAUSE_TIME = 1


In [4]:
viewed_pages = set()

queue = [
    'https://wallpaperplay.com/board/4k-minimalist-wallpapers'
]

In [5]:
def scroll_to_bottom(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [6]:
driver.get('https://wallpaperplay.com/')
scroll_to_bottom(driver)
categories = driver.find_elements(By.XPATH, '//a[@class="ui fluid image"]')
for cat in categories:
    cat_url = cat.get_attribute('href')
    eval_val = 0
        
    for whitelist_word in whitelist:
        if whitelist_word in cat_url:
            eval_val += 1
    for blacklist_word in blacklist:
        if blacklist_word in cat_url:
            eval_val -= 1
    if eval_val > 0:
        queue.insert(0, cat_url)
    else:
        queue.append(cat_url)
        
print(len(queue))

with open('/data1/wallpaperPlayQueue.pkl', 'wb') as f:
    pickle.dump(queue, f, protocol=pickle.HIGHEST_PROTOCOL)

5529


In [17]:
with open('/data1/wallpaperPlayQueue.pkl', 'rb') as f:
    queue = pickle.load(f)
print(len(queue))

5529


In [25]:
root_url = 'https://wallpaperplay.com/'
data_path = pathlib.Path('/data1', 'Raw', 'WallpaperPlay')
data_directories = [f for f in data_path.iterdir() if f.is_dir()]
print(len(data_directories))

# d = data_directories[0]
# print('{}board/{}'.format(root_url, str(d).split('/')[-1]))

viewed_pages = set(['{}board/{}'.format(root_url, str(d).split('/')[-1]) for d in data_directories])

939


In [28]:
queue = [link for link in queue if link not in viewed_pages]
print(len(queue))

print(queue[0])
print(list(viewed_pages)[0])



4589
https://wallpaperplay.com/board/evil-christmas-wallpapers
https://wallpaperplay.com/board/future-wallpapers


In [6]:
with open('/data1/wallpaperPlayQueueCurrent.pkl', 'rb') as f:
    queue = pickle.load(f)
print(len(queue))

3609


In [7]:
i=0

In [9]:
# page_limit = 200*10

while True:
#     if i > page_limit:
#         break
    
    if len(queue) <= 0:
        break
        
    url = queue.pop(0)
    
    if url in viewed_pages:
        continue
    
    driver.get(url)
    
    image_category = url.split('/')[-1]
    
    cat_path = pathlib.Path('/data1', 'Raw', 'WallpaperPlay', image_category)
    if cat_path.is_dir():
        viewed_pages.add(url)
        print('skipped', url)
        time.sleep(WEBSITE_PAUSE_TIME)
        continue
    
    time.sleep(WEBSITE_PAUSE_TIME)
    scroll_to_bottom(driver)
    time.sleep(WEBSITE_PAUSE_TIME)
    
    with open('/data1/wallpaperPlayQueueCurrent.pkl', 'wb') as f:
        pickle.dump(queue, f, protocol=pickle.HIGHEST_PROTOCOL)
    
    imgs = driver.find_elements(By.XPATH, '//img[@class="isWiden thumb ads_popup"]')
    
    
    
    print(url, len(imgs))
    
    for img in imgs:
        img_url = img.get_attribute('data-download')
        
        img_id = img_url.split('/')[-1]
        path = pathlib.Path('/data1', 'Raw', 'WallpaperPlay', image_category, '{}.jpg'.format(img_id))
        path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(path,'wb') as f:
            f.write(requests.get(img_url).content)
            f.close()
        
        time.sleep(IMAGE_PAUSE_TIME)
        
        i+=1
        
    print(i, '{} left'.format(len(queue)))
        
#     related_wallpapers = driver.find_elements(By.XPATH, '//div[@class="column collection_thumb"]/a')
    
#     for link in related_wallpapers:
#         wp_link = link.get_attribute('href')
#         eval_val = 0
        
#         for whitelist_word in whitelist:
#             if whitelist_word in wp_link:
#                 eval_val += 1
#         for blacklist_word in blacklist:
#             if blacklist_word in wp_link:
#                 eval_val -= 1
#         if eval_val > 0:
#             queue.insert(0, wp_link)
#         else:
#             queue.append(wp_link)
        
    viewed_pages.add(url)
    
    
            
    time.sleep(WEBSITE_PAUSE_TIME)

https://wallpaperplay.com/board/pretty-phone-wallpapers 3
18322 3292 left
https://wallpaperplay.com/board/cool-egyptian-wallpapers 57
18379 3291 left
https://wallpaperplay.com/board/vegeta-phone-wallpapers 44
18423 3290 left
https://wallpaperplay.com/board/mustang-iphone-wallpapers 36
18459 3289 left
https://wallpaperplay.com/board/slammed-car-wallpapers 61
18520 3288 left
https://wallpaperplay.com/board/guitar-girls-wallpapers 76
18596 3287 left
https://wallpaperplay.com/board/green-biohazard-wallpapers 64
18660 3286 left
https://wallpaperplay.com/board/antique-looking-wallpapers 55
18715 3285 left
https://wallpaperplay.com/board/farm-scene-wallpapers 42
18757 3284 left
https://wallpaperplay.com/board/cool-faze-wallpapers 71
18828 3283 left
https://wallpaperplay.com/board/manga-wallpapers 67
18895 3282 left
https://wallpaperplay.com/board/beautiful-rainbow-wallpapers 60
18955 3281 left
https://wallpaperplay.com/board/sugar-skull-wallpapers 48
19003 3280 left
https://wallpaperplay.com/

24745 3182 left
https://wallpaperplay.com/board/1080p-space-wallpapers 77
24822 3181 left
https://wallpaperplay.com/board/fantasy-princess-wallpapers 70
24892 3180 left
https://wallpaperplay.com/board/clone-wars-wallpapers 61
24953 3179 left
https://wallpaperplay.com/board/personal-photo-wallpapers 73
25026 3178 left
https://wallpaperplay.com/board/arthas-wallpapers 67
25093 3177 left
https://wallpaperplay.com/board/red-hd-wallpapers 75
25168 3176 left
https://wallpaperplay.com/board/beautiful-smile-wallpapers 57
25225 3175 left
https://wallpaperplay.com/board/ewok-wallpapers 59
25284 3174 left
https://wallpaperplay.com/board/3d-fairy-wallpapers 59
25343 3173 left
https://wallpaperplay.com/board/zoroark-wallpapers 60
25403 3172 left
https://wallpaperplay.com/board/black-flag-wallpapers 75
25478 3171 left
https://wallpaperplay.com/board/rockabilly-girls-wallpapers 42
25520 3170 left
https://wallpaperplay.com/board/flash-superhero-wallpapers 57
25577 3169 left
https://wallpaperplay.com/b

https://wallpaperplay.com/board/3d-illusion-wallpapers 59
31169 3071 left
https://wallpaperplay.com/board/army-acu-wallpapers 50
31219 3070 left
https://wallpaperplay.com/board/wizards-wallpapers 72
31291 3069 left
https://wallpaperplay.com/board/river-scene-wallpapers 55
31346 3068 left
https://wallpaperplay.com/board/piccolo-wallpapers 55
31401 3067 left
https://wallpaperplay.com/board/green-nebula-wallpapers 68
31469 3066 left
https://wallpaperplay.com/board/good-friday-wallpapers 65
31534 3065 left
https://wallpaperplay.com/board/movies-wallpapers 67
31601 3064 left
https://wallpaperplay.com/board/cute-infinity-wallpapers 44
31645 3063 left
https://wallpaperplay.com/board/sea-life-wallpapers 66
31711 3062 left
https://wallpaperplay.com/board/hawaiian-beaches-wallpapers 57
31768 3061 left
https://wallpaperplay.com/board/impressionism-wallpapers 49
31817 3060 left
https://wallpaperplay.com/board/sedona-arizona-wallpapers 50
31867 3059 left
https://wallpaperplay.com/board/bruins-phone

37739 2961 left
https://wallpaperplay.com/board/baby-cheetah-wallpapers 61
37800 2960 left
https://wallpaperplay.com/board/retina-display-wallpapers 73
37873 2959 left
https://wallpaperplay.com/board/light-pink-wallpapers 57
37930 2958 left
https://wallpaperplay.com/board/best-pokemon-wallpapers 64
37994 2957 left
https://wallpaperplay.com/board/batman-family-wallpapers 67
38061 2956 left
https://wallpaperplay.com/board/cool-pink-wallpapers 58
38119 2955 left
https://wallpaperplay.com/board/wedding-dress-wallpapers 45
38164 2954 left
https://wallpaperplay.com/board/kawaii-halloween-wallpapers 32
38196 2953 left
https://wallpaperplay.com/board/swim-wallpapers 43
38239 2952 left
https://wallpaperplay.com/board/woodworking-wallpapers 57
38296 2951 left
https://wallpaperplay.com/board/million-dollar-wallpapers 67
38363 2950 left
https://wallpaperplay.com/board/evil-halloween-wallpapers 65
38428 2949 left
https://wallpaperplay.com/board/baby-orangutan-wallpapers 65
38493 2948 left
https://w

KeyboardInterrupt: 

In [ ]:
with open('/data1/wallpaperPlayPhotos.pkl', 'wb') as f:
    pickle.dump(image_urls, f)

with open('/data1/wallpaperPlayPhotoCategories.pkl', 'wb') as f:
    pickle.dump(image_categories, f)